<a href="https://colab.research.google.com/github/Priyansh-Kedia/AmazonChallenge/blob/master/AmazonChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Add all imports here

from nltk.corpus import stopwords
import nltk
import string
import re
from collections import Counter
import math
import pickle

import pandas as pd
import csv

from keras.preprocessing.text import Tokenizer

import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.preprocessing import image
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dropout, Flatten
from keras import layers

In [ ]:
# Download and unzip dataset

!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
!unzip dataset52a7b21.zip

--2021-07-30 19:50:44--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.128.106
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.128.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  14.0MB/s    in 76s     

2021-07-30 19:52:01 (13.3 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


In [3]:
# Download nltk stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Import the dataset in pandas

df_ = pd.read_csv("dataset/train.csv", low_memory=True, escapechar="\\", quoting=csv.QUOTE_NONE)

df = df_.head(50000)

In [5]:
# Add the column variables here

df_ = None

TITLE = "TITLE"
PRODUCT_ID = "PRODUCT_ID"
DESCRIPTION = "DESCRIPTION"
BULLET_POINTS = "BULLET_POINTS"
BRAND = "BRAND"
BROWSE_NODE_ID = "BROWSE_NODE_ID"

SIZE = 50000

In [6]:
# Load the doc
def load_doc(df):
  titles = ' '.join(df[TITLE].fillna('').astype(str))
  descriptions = ' '.join(df[DESCRIPTION].fillna('').astype(str))
  bullets = ' '.join(df[BULLET_POINTS].fillna('').astype(str))

  return (titles + descriptions + bullets)

In [7]:
# To remove html tags

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>') # replace with re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') for more strictness
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [8]:
def clean_doc(doc):
  doc = cleanhtml(doc)
  tokens = doc.split() # Add delimiter if needed

  punctuations = str.maketrans('','', string.punctuation)

  tokens = [w.translate(punctuations) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words("english"))
  #print("stop", stop_words)
  tokens = [w for w in tokens if not w.casefold() in stop_words]

  tokens = [word for word in tokens if len(word) > 1]

  return tokens


In [9]:
# Create, Store, and Read Hashmap functions

def create_hashmap(dataframe):
    from collections import defaultdict
    Hashmap = defaultdict(list)

    for i, [brand, id] in enumerate(dataframe.values[:, [3, 4]]):
        if int(id) not in Hashmap[str(brand)]:
            Hashmap[str(brand)].append(int(id))

    return Hashmap

def store_hashmap(Hashmap, filename="Hashmap"):
    
    with open(filename + '.pickle', 'wb') as handle:
        pickle.dump(Hashmap, handle)

def read_hashmap(filename="Hashmap"):
    
    with open(filename + '.pickle', 'rb') as handle:
        Hashmap = pickle.load(handle)

    return Hashmap

In [10]:
# Add words to vocab

def add_doc_to_vocab(df, vocab):
  doc = load_doc(df)
  tokens = clean_doc(doc)
  vocab.update(tokens)

In [11]:
# Save the vocab

def save_list(lines, filename):
    # convert the lines into single blob of text
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [12]:
# create the vocab

vocab = Counter()
add_doc_to_vocab(df, vocab)

# print(vocab.most_common(50))

In [13]:
# Save vocab according to occurance

min_occurance = 5
tokens = [k for k,c in vocab.items() if c >= min_occurance]

save_list(tokens, "vocab.txt")

In [14]:
# def create_splits():
#   n = len(df)
#   train_factor = math.ceil(n*0.7)

#   X_train = df[:train_factor]
#   X_Validation = df[train_factor:]

#   return X_train, X_Validation


In [15]:
#Create SENTENCE by combining TITLE, DESCRIPTIONS, BULLET POINTS
def create_sentences(df):

  df[BULLET_POINTS] = df[BULLET_POINTS].str.strip('[]').astype(str)
  df['SENTENCES'] = df[df.columns[0:3]].apply(lambda x: ', '.join(x.fillna('').astype(str)),axis=1)
  df.SENTENCES = df.SENTENCES.astype(str)
  df['SENTENCES'] = df.apply(lambda row: clean_doc(row.SENTENCES), axis=1)
  df['SENTENCES'] = df['SENTENCES'].apply(lambda x: ' '.join(map(str, x)))


  return df

df1 = create_sentences(df)


In [16]:
def open_vocab(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text


In [17]:
vocab = open_vocab("vocab.txt")
vocab = vocab.split()
vocab = set(vocab)
print(vocab)

{'WildHorn', 'Attractive', 'SpouseFriends', 'sheesham', 'Keyring', 'Length', 'Wilson', 'Availability', 'Fig', 'decoder', 'tasking', 'Sander', 'authorize', 'botanicals', 'trap', 'Pandora', 'Christ', 'prone', 'Creatively', 'featherweight', 'Hynix', 'omen', 'Bro', 'setMATERIAL', 'fenugreek', 'BRANDONN', 'situated', 'breakageScratch', 'scratchesdamages', 'earthenware', 'CATV', 'Brave', 'eyeliner', 'doubledeck', 'pioneers', 'cultivation', 'pests', 'segments', 'Experienced', 'Australia', 'Pullon', 'VERO', 'terms', 'Quarts', 'doubles', 'Lugs', 'SIDDHI', 'coordinated', 'Stonish', 'dispatched', 'ServicenbspPrompt', 'retirements', 'Roku', 'MANTO', 'frets', 'vision', 'pocketsThis', 'canteen', 'electrodes', 'Zoey', 'Beta', 'Excalibur', 'Antichapping', 'fragile', 'Samsung', 'experiments', 'fullcolor', 'locket', 'Cuffs', 'Octova', 'CottonWith', 'stringing', 'Eyebrow', 'tunic', 'fullcoverage', 'bleachfree', 'architectural', 'Labrador', 'preserve', 'chikan', 'Onn', 'Proffisy', 'BlendSize', 'spikes', '

In [18]:
sentences = []
text = []
coloumns = list = (df1['SENTENCES'])
for i in coloumns:
  text = i.split(" ")
  text = [x for x in text if x in vocab]
  res = []
  for i in text:
    if i not in res:
        res.append(i)
  res = ' '.join(res)
  sentences.append(res)

sentencess = pd.DataFrame(sentences)
df1["SENTENCES"] = sentencess

sentencess = None



In [19]:
# Create a tokenizer to make embeddings

tokenizer = Tokenizer()
docs = sentences

tokenizer.fit_on_texts(docs)
sentences = None
docs = None

X_train, X_val, Y_train, Y_val = train_test_split(df1["SENTENCES"], df1[BROWSE_NODE_ID], test_size=0.33)

X_train = tokenizer.texts_to_matrix(X_train, mode='freq')
X_val = tokenizer.texts_to_matrix(X_val, mode='freq')


In [20]:
"""
binary, count, freq, tfidf
"""
# X_train = tokenizer.texts_to_matrix(docs, mode='freq')
# Y_train = df[BROWSE_NODE_ID].head(100)
# print(Y_train)



'\nbinary, count, freq, tfidf\n'

In [ ]:
input_layer = keras.Input(shape=X_train.shape[1], name="INPUT")
hidden_layer_1 = layers.Dense(256, activation='relu', name="HIDDEN_1")(input_layer)
code_layer = layers.Dense(256, activation='relu', name="CODE")(hidden_layer_1)
output_layer = layers.Dense(1, activation='relu', name="OUTPUT")(code_layer)
AE = Model(input_layer, output_layer)
AE.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
AE.summary()
history = AE.fit(X_train, Y_train, epochs=50 , batch_size= 2000, validation_data= (X_Validation,Y_Validation), verbose = 2)